In [225]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_rows', None)     # show all rows
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option('display.width', None)        # no fixed width wrapping
pd.set_option('display.max_colwidth', None) # show full cell content

In [226]:
def peek_df(df, target_col=None):
    print("="*40)
    print("HEAD OF DATAFRAME")
    print("="*40)
    print(df.head(), "\n")

    print("="*40)
    print("SHAPE OF DATAFRAME")
    print("="*40)
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}\n")

    print("="*40)
    print("INFO")
    print("="*40)
    df.info()
    print()

    print("="*40)
    print("MISSING VALUES")
    print("="*40)
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if not missing.empty:
        missing_percent = (missing / len(df)) * 100
        missing_df = pd.DataFrame({
            "Missing Values": missing,
            "Percent": missing_percent.round(2)
        })
        print(missing_df, "\n")
    else:
        print("No missing values.\n")

    print("="*40)
    print("DESCRIPTIVE STATS (NUMERIC)")
    print("="*40)
    print(df.describe(), "\n")

    print("="*40)
    print("DESCRIPTIVE STATS (CATEGORICAL)")
    print("="*40)
    categorical_cols = df.select_dtypes(include='object').columns
    if len(categorical_cols) > 0:
        print(df[categorical_cols].describe(), "\n")
    else:
        print("No categorical (object) columns to describe.\n")

    print("="*40)
    print("UNIQUE VALUES")
    print("="*40)
    for col in df.columns:
        if df[col].nunique(dropna=True) <= 20:
            print(f"{col}: {df[col].unique()}")

    if target_col:
        print("="*40)
        print(f"TARGET COLUMN DISTRIBUTION: {target_col}")
        print("="*40)
        print(df[target_col].value_counts(), "\n")

        sns.countplot(x=target_col, data=df)
        plt.title(f"Countplot of '{target_col}'")
        plt.show()

        print("="*40)
        print("CORRELATION HEATMAP (NUMERIC FEATURES)")
        print("="*40)
        numeric_df = df.select_dtypes(include=['number'])
        sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Correlation Heatmap")
        plt.show()

In [227]:
# Read each gw and stack them vertically
dfs = []
for gw_num in range(1, 39):
    df_this_gw = pd.read_csv(f'gw{gw_num}.csv')
    dfs.append(df_this_gw)

df = pd.concat(dfs, ignore_index=True)

In [228]:
# drop all players who don't have 38 entries
df = df.groupby('element').filter(lambda x: len(x) == 38).copy()

In [229]:
def reassign_rounds(df_this):
    updated_rows = []

    for player_id, group in df_this.groupby('element'):
        group_sorted = group.sort_values('kickoff_time').copy()
        group_sorted['round'] = list(range(1, 39))
        updated_rows.append(group_sorted)

    return pd.concat(updated_rows, ignore_index=True)

In [230]:
# fix double gameweeks
df = reassign_rounds(df).copy()

In [231]:
# drop all "mng_"  columns in Vaastav dataset
df = df.loc[:,~df.columns.str.startswith('mng_')].copy()

In [232]:
# drop all "transfers_" columns in Vaastav dataset
df = df.loc[:,~df.columns.str.startswith('transfers_')].copy()

In [233]:
# drop more columns in Vaastav dataset
df = df.drop(columns=["modified", "kickoff_time", "selected"])

In [234]:
peek_df(df)

HEAD OF DATAFRAME
                    name position     team   xP  assists  bonus  bps  \
0  Fábio Ferreira Vieira      MID  Arsenal  2.3        0      0    0   
1  Fábio Ferreira Vieira      MID  Arsenal  0.8        0      0    0   
2  Fábio Ferreira Vieira      MID  Arsenal  0.0        0      0    0   
3  Fábio Ferreira Vieira      MID  Arsenal  0.0        0      0    0   
4  Fábio Ferreira Vieira      MID  Arsenal  0.0        0      0    0   

   clean_sheets  creativity  element  expected_assists  \
0             0         0.0        1               0.0   
1             0         0.0        1               0.0   
2             0         0.0        1               0.0   
3             0         0.0        1               0.0   
4             0         0.0        1               0.0   

   expected_goal_involvements  expected_goals  expected_goals_conceded  \
0                         0.0             0.0                      0.0   
1                         0.0             0.0       

In [235]:
# Columns to keep as single columns (non-pivoted)
player_info_cols = ['element', 'name', 'team', 'position', 'round']

# Extract player info — keep first occurrence per element
player_info = df[player_info_cols].drop_duplicates(subset='element')

# Drop managers
player_info = player_info[player_info['position'] != 'AM']

# fix indices
player_info = player_info.set_index('element')
player_info = player_info.sort_index()

In [236]:
player_info.head()

,name,team,position,round
element,,,,
1,Fábio Ferreira Vieira,Arsenal,MID,1
2,Gabriel Fernando de Jesus,Arsenal,FWD,1
3,Gabriel dos Santos Magalhães,Arsenal,DEF,1
4,Kai Havertz,Arsenal,FWD,1
5,Karl Hein,Arsenal,GK,1


In [237]:
# Columns to pivot (exclude player info + 'round' and 'element')
cols_to_pivot = [col for col in df.columns if col not in player_info_cols + ['round']]

In [238]:
df.dtypes

name                           object
position                       object
team                           object
xP                            float64
assists                         int64
bonus                           int64
bps                             int64
clean_sheets                    int64
creativity                    float64
element                         int64
expected_assists              float64
expected_goal_involvements    float64
expected_goals                float64
expected_goals_conceded       float64
fixture                         int64
goals_conceded                  int64
goals_scored                    int64
ict_index                     float64
influence                     float64
minutes                         int64
opponent_team                   int64
own_goals                       int64
penalties_missed                int64
penalties_saved                 int64
red_cards                       int64
round                           int64
saves       

In [239]:
# Pivot gameweek data: index=element, columns=round, values=cols_to_pivot
pivoted = df.pivot(
    index='element',
    columns='round',
    values=cols_to_pivot
)

In [240]:
# Flatten MultiIndex columns to 'gwX_colname'
pivoted.columns = [f"gw{int(gw)}_{col}" for col, gw in pivoted.columns]

# Combine pivoted data with player info (single columns)
final_df = player_info.join(pivoted).reset_index()

# drop "round" column
final_df = final_df.drop(columns=['round'])

In [241]:
final_df.columns[final_df.columns.str.startswith('gw38')]

Index(['gw38_xP', 'gw38_assists', 'gw38_bonus', 'gw38_bps',
       'gw38_clean_sheets', 'gw38_creativity', 'gw38_expected_assists',
       'gw38_expected_goal_involvements', 'gw38_expected_goals',
       'gw38_expected_goals_conceded', 'gw38_fixture', 'gw38_goals_conceded',
       'gw38_goals_scored', 'gw38_ict_index', 'gw38_influence', 'gw38_minutes',
       'gw38_opponent_team', 'gw38_own_goals', 'gw38_penalties_missed',
       'gw38_penalties_saved', 'gw38_red_cards', 'gw38_saves', 'gw38_starts',
       'gw38_team_a_score', 'gw38_team_h_score', 'gw38_threat',
       'gw38_total_points', 'gw38_value', 'gw38_was_home',
       'gw38_yellow_cards'],
      dtype='object')

In [242]:
# get back numeric columns
numeric_cols = [col for col in final_df.columns if col.startswith("gw")]

# Convert to numeric, coercing errors (non-numeric strings become NaN)
final_df[numeric_cols] = final_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [243]:
peek_df(final_df)

HEAD OF DATAFRAME
   element                          name     team position  gw1_xP  gw2_xP  \
0        1         Fábio Ferreira Vieira  Arsenal      MID     2.3     0.8   
1        2     Gabriel Fernando de Jesus  Arsenal      FWD     3.2     1.0   
2        3  Gabriel dos Santos Magalhães  Arsenal      DEF     4.2     7.0   
3        4                   Kai Havertz  Arsenal      FWD     3.5     8.0   
4        5                     Karl Hein  Arsenal       GK     0.0     0.0   

   gw3_xP  gw4_xP  gw5_xP  gw6_xP  gw7_xP  gw8_xP  gw9_xP  gw10_xP  gw11_xP  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0      0.0   
1     0.0     1.2     0.2     2.2     2.3     2.0     0.7      1.7      1.2   
2     5.7     8.2     7.5     9.2     7.7     2.3     1.3      1.7      2.2   
3     8.3     7.0     3.5     4.8     6.0     4.7     4.0      2.7      1.4   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0      0.0   

   gw12_xP  gw13_xP  gw14_xP  gw15_xP 

In [244]:
final_df.columns.to_list()

['element',
 'name',
 'team',
 'position',
 'gw1_xP',
 'gw2_xP',
 'gw3_xP',
 'gw4_xP',
 'gw5_xP',
 'gw6_xP',
 'gw7_xP',
 'gw8_xP',
 'gw9_xP',
 'gw10_xP',
 'gw11_xP',
 'gw12_xP',
 'gw13_xP',
 'gw14_xP',
 'gw15_xP',
 'gw16_xP',
 'gw17_xP',
 'gw18_xP',
 'gw19_xP',
 'gw20_xP',
 'gw21_xP',
 'gw22_xP',
 'gw23_xP',
 'gw24_xP',
 'gw25_xP',
 'gw26_xP',
 'gw27_xP',
 'gw28_xP',
 'gw29_xP',
 'gw30_xP',
 'gw31_xP',
 'gw32_xP',
 'gw33_xP',
 'gw34_xP',
 'gw35_xP',
 'gw36_xP',
 'gw37_xP',
 'gw38_xP',
 'gw1_assists',
 'gw2_assists',
 'gw3_assists',
 'gw4_assists',
 'gw5_assists',
 'gw6_assists',
 'gw7_assists',
 'gw8_assists',
 'gw9_assists',
 'gw10_assists',
 'gw11_assists',
 'gw12_assists',
 'gw13_assists',
 'gw14_assists',
 'gw15_assists',
 'gw16_assists',
 'gw17_assists',
 'gw18_assists',
 'gw19_assists',
 'gw20_assists',
 'gw21_assists',
 'gw22_assists',
 'gw23_assists',
 'gw24_assists',
 'gw25_assists',
 'gw26_assists',
 'gw27_assists',
 'gw28_assists',
 'gw29_assists',
 'gw30_assists',
 'gw31_as

In [246]:
final_df.to_csv('fpl_data_all_gw.csv', index=False)

In [247]:
final_df.shape

(615, 1144)